<a href="https://colab.research.google.com/github/IsaacFigNewton/Text-to-Timeline/blob/main/Sample_Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Download Packages

In [ ]:
# Broken libraries I've tried: AllenNLP, Coreferee, neuralcoref, spacy-experimental, stanford-corenlp (only api available)
!pip install fastcoref
!pip install intervaltree

In [ ]:
!git clone https://github.com/IsaacFigNewton/Text-to-Timeline.git

# Import Libraries

In [ ]:
import sys
import os
import json
import spacy
import networkx as nx
import matplotlib.pyplot as plt

In [ ]:
from fastcoref import FCoref, spacy_component

In [ ]:
# Add the timeline reconstruction modules directory to the Python path
module_path = os.path.abspath('./Text-to-Timeline/modules')
if module_path not in sys.path:
    sys.path.append(module_path)

from pipeline import *
from utils import plot_graph_from_edge_list

In [ ]:
default_nlp_model = spacy.load("en_core_web_sm")

fastcoref_model = FCoref()

coref_resolution_model = spacy.load("en_core_web_sm", exclude=["parser", "lemmatizer", "ner", "textcat"])
coref_resolution_model.add_pipe("fastcoref")

# Config

In [ ]:
tests = [
    "The frog jumped over the goose. Mr. Holmes is gay. Then the frog fell into the abyss. The goose followed the frog into the abyss and after that ate a different frog.",
    # "Although he was very busy with his work, Peter had had enough of it. He and his wife decided they needed a holiday. They travelled to Spain because they loved the country very much.",
    # "John met Paul after he finished work. He suggested they grab a drink.",
    # "The book was on the table when Sarah handed it to Mary. She smiled and thanked her.",
    # "Tom told Jerry that he had failed the exam.",
    # "The city council refused the demonstrators a permit because they feared violence.",
    # "Anna told Lucy that her idea was brilliant.",
    # "The scientist interviewed the assistant while she was setting up the experiment.",
    # "After the dog bit the man, he ran away.",
    # "The mechanic fixed the car while it was raining. He was soaked by the end.",
    # "David thanked Michael after his birthday party.",
    # "Karen lent her book to Julie because she needed it for class."
]

In [ ]:
default_nlp_model = spacy.load("en_core_web_sm")

fastcoref_model = FCoref()

coref_resolution_model = spacy.load("en_core_web_sm", exclude=["parser", "lemmatizer", "ner", "textcat"])
coref_resolution_model.add_pipe("fastcoref")

# Helper functions

In [ ]:
def run_tests(texts:str):
  ambiguated_texts = [[t, None] for t in texts]
  ambiguated_test = None

  for i in range(len(texts)):
    _, ambiguated_test = ambiguate_text(texts[i])
    ambiguated_texts[i][1] = ambiguated_test

  for test in ambiguated_texts:
    print(f"original:\n{test[0]}")
    print(f"disambiguated:\n{test[1]}")
    # print()

# Tests

In [ ]:
test_infos = []
for test in tests:
  test_info = get_text_info_json(text=test,
                                 nlp_model=default_nlp_model,
                                 fastcoref_model=fastcoref_model,
                                 coref_resolution_model=coref_resolution_model)
  test_infos.append(test_info)

In [ ]:
plot_graph_from_edge_list(test_infos[0]["edges"])